In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import math
from tqdm import tqdm
from ast import literal_eval
import pickle
import time
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [26]:
# ---------------------
# Load T5-Base Model
# ---------------------
summarizer_model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

summarizer_model.eval()
for param in summarizer_model.parameters():
    param.requires_grad = False

In [27]:
# Load nid2body from pickle
with open("nid2body.pkl", "rb") as f:
    nid2body = pickle.load(f)

# Debug print
print(f"✅ Loaded nid2body with {len(nid2body)} items")
sample_nid = list(nid2body.keys())[0]
print(f"🧾 Sample NID: {sample_nid}\n📝 Headline: {nid2body[sample_nid][:300]}")

✅ Loaded nid2body with 113762 items
🧾 Sample NID: N10000
📝 Headline: Predicting Atlanta United's lineup against Columbus Crew in the U.S. Open Cup


In [28]:
# Load sid2sum from pickle
with open("sid2sum.pkl", "rb") as f:
    sid2sum = pickle.load(f)

# Debug print
print(f"✅ Loaded sid2sum with {len(sid2sum)} items")
sample_sid = list(sid2sum.keys())[0]
print(f"🧾 Sample SID: {sample_sid}\n📝 Summary: {sid2sum[sample_sid][:300]}")

✅ Loaded sid2sum with 135001 items
🧾 Sample SID: S-1
📝 Summary: The officer reportedly also pointed his gun at Harper and her children.


In [29]:
# === Device and Precision Setup ===
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 768

# === Utility Functions ===
def get_embedding(key, table, dim):
    if key not in table:
        table[key] = torch.nn.Parameter(torch.randn(dim, dtype=torch.float32, device=device) * 0.01, requires_grad=True)
    return table[key]


# === Load Embeddings ===
with open("summary_T5.pkl", "rb") as f:
    summary_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("newsbody_T5.pkl", "rb") as f:
    newsbody_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("headline_T5.pkl", "rb") as f:
    headline_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}

embed_tables = {
    'summary': summary_embed,
    'newsbody': newsbody_embed,
    'headline': headline_embed
}

# === Load Dataset ===
lookup_df = pd.read_csv("w2p_engage_list.csv").set_index('EdgeID')
train_df = pd.read_csv("train_w2p.csv")

In [79]:
train_df=train_df[:50000]
train_df

,Unnamed: 0,UserID,EHist,EPos
0,2,U10000_1,"['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8...",E84
1,3,U10000_2,"['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8...",E133
2,11,U100006_1,['E151'],E152
3,12,U100006_2,"['E151', 'E152', 'E153', 'E154', 'E155', 'E156...",E168
4,13,U100006_3,"['E151', 'E152', 'E153', 'E154', 'E155', 'E156...",E230
...,...,...,...,...
49995,195583,U233775_3,"['E1882808', 'E1882809', 'E1882810', 'E1882811...",E1882859
49996,195584,U233775_4,"['E1882808', 'E1882809', 'E1882810', 'E1882811...",E1882866
49997,195585,U233775_5,"['E1882808', 'E1882809', 'E1882810', 'E1882811...",E1882869
49998,195586,U233775_6,"['E1882808', 'E1882809', 'E1882810', 'E1882811...",E1882920


In [31]:
lookup_df

,Unnamed: 0,Head,Relation,Tail,User
EdgeID,,,,,
E1,0,U10000,skip,N110699,U10000
E2,1,N110699,skip,N104733,U10000
E3,2,N104733,skip,N80645,U10000
E4,3,N80645,skip,N76869,U10000
E5,4,N76869,skip,N119531,U10000
...,...,...,...,...,...
E18413173,18413172,N107780,click,N97051,U249644
E18413174,18413173,N97051,click,N81002,U249644
E18413175,18413174,N81002,click,N11725,U249644


In [ ]:
# # ---------------------
# # Build Tail2Idx
# # ---------------------
# tail_set = set()

# print("Building Tail2Idx from EHist and EPos...")
# for row in tqdm(train_df.itertuples(), total=len(train_df), desc="Collecting tails"):
#     try:
#         bhist = literal_eval(row.EHist)
#         bpos = row.EPos

#         # Add tails from Bhist
#         for b_id in bhist:
#             if b_id in lookup_df.index:
#                 tail = lookup_df.loc[b_id, 'Tail']
#                 tail_set.add(tail)

#         # Add tail from Bpos
#         if bpos in lookup_df.index:
#             tail = lookup_df.loc[bpos, 'Tail']
#             tail_set.add(tail)

#     except Exception as e:
#         print(f"[Skip] Error in row {row.Index}: {e}")

# # === Final mappings ===
# tail_ids = sorted(tail_set)
# tail2idx = {tid: idx for idx, tid in enumerate(tail_ids)}
# idx2tail = {idx: tid for tid, idx in tail2idx.items()}

# print(f"✅ Tail2Idx built with {len(tail2idx)} unique tail IDs.")

# # ---------------------
# # Save as pickle
# # ---------------------
# with open("tail_mappings.pkl", "wb") as f:
#     pickle.dump({"tail2idx": tail2idx, "idx2tail": idx2tail}, f)

# print("💾 Saved tail2idx and idx2tail to tail_mappings.pkl")


In [32]:
# ---------------------
# Load back from pickle
# ---------------------
with open("tail_mappings.pkl", "rb") as f:
    mappings = pickle.load(f)

tail2idx = mappings["tail2idx"]
idx2tail = mappings["idx2tail"]

print(f"✅ Loaded tail2idx with {len(tail2idx)} entries")
print(f"🧾 Sample tail → idx: {list(tail2idx.items())[:3]}")
print(f"🧾 Sample idx → tail: {list(idx2tail.items())[:3]}")

✅ Loaded tail2idx with 544137 entries
🧾 Sample tail → idx: [('N10000', 0), ('N100001', 1), ('N100003', 2)]
🧾 Sample idx → tail: [(0, 'N10000'), (1, 'N100001'), (2, 'N100003')]


In [42]:
# -------------------------
# Sequence Engagement/Behavior Encoder
# -------------------------

class BehaviorEncoder(nn.Module):
    def __init__(self, hidden_dim, tail2idx, device, debug=False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.debug = debug
        self.tail2idx = tail2idx

        # === Base action vectors ===
        self.e_clk = nn.Parameter(torch.tensor([1., 0., 0., 0.], device=device))
        self.e_skp = nn.Parameter(torch.tensor([0., 1., 0., 0.], device=device))
        self.e_gensumm = nn.Parameter(torch.tensor([0., 0., 1., 0.], device=device))
        self.e_sumgen = nn.Parameter(torch.tensor([0., 0., 0., 1.], device=device))

        # Action-specific transforms
        self.W_clk = nn.Linear(4, hidden_dim, bias=False)
        self.W_skp = nn.Linear(4, hidden_dim, bias=False)
        self.W_gensumm = nn.Linear(4, hidden_dim, bias=False)
        self.W_sumgen = nn.Linear(4, hidden_dim, bias=False)

        # State transforms
        self.W_pull = nn.Linear(1, hidden_dim, bias=False)
        self.W_s = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_d = nn.Linear(hidden_dim, hidden_dim, bias=False)

        # Fusion
        self.Wh = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.Wc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.Wz = nn.Linear(hidden_dim, 3, bias=False)
        self.b_z = nn.Parameter(torch.zeros(3, device=device))
        self.W_emb = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.b_emb = nn.Parameter(torch.zeros(hidden_dim, device=device))

        # Rotation/translation
        self.W_angle = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_theta = nn.Linear(hidden_dim, 1, bias=False)
        self.W_h = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_m = nn.Linear(hidden_dim, 1, bias=False)

        # Scalars
        self.alpha = nn.Parameter(torch.tensor(0.5, device=device))
        self.beta = nn.Parameter(torch.tensor(0.5, device=device))

        # Classifier head over tails
        self.classifier = nn.Linear(hidden_dim, len(tail2idx))

        # Next-step prediction head
        self.W_next = nn.Linear(hidden_dim, hidden_dim, bias=False)

    def _show(self, name, tensor, maxlen=6):
        if not self.debug: 
            return
        if isinstance(tensor, torch.Tensor):
            flat = tensor.detach().cpu().numpy().flatten()
            vals = ", ".join(f"{x:.4f}" for x in flat[:maxlen])
            if len(flat) > maxlen: vals += ", ..."
            print(f"{name} (shape={tuple(tensor.shape)}): [{vals}]")
        else:
            print(f"{name}: {tensor}")

    def softmin_pool(self, a, b):
        return -self.alpha * torch.log(torch.exp(a / self.alpha) +
                                       torch.exp(b / self.alpha) + 1e-9)

    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables):
        total_loss = torch.tensor(0., dtype=torch.float32, device=self.device)

        # === PASS 1: raw step embeddings (E_seq) ===
        E_seq = []
        h_clk = torch.zeros(self.hidden_dim, device=self.device)
        h_skp = torch.zeros(self.hidden_dim, device=self.device)
        h = torch.zeros(self.hidden_dim, device=self.device)

        for t, b_id in enumerate(Bhist):
            if b_id not in lookup_df.index:
                continue
            row = lookup_df.loc[b_id]
            tail_id, rel = row['Tail'], row['Relation']

            d_i = embed_tables['newsbody'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))
            s_i = embed_tables['summary'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))
            d_i_title = embed_tables['headline'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))

            # init state
            if t == 0:
                head_emb = embed_tables['headline'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))
                h_clk, h_skp = head_emb, head_emb
                h = torch.sigmoid(self.W_s(head_emb)) * h_clk + (1 - torch.sigmoid(self.W_s(head_emb))) * h_skp

            # relation-specific context c_i
            if rel == "click":
                c_i = (self.W_clk.weight @ self.e_clk * h) * d_i
            elif rel == "skip":
                d_ip1 = torch.zeros_like(d_i)
                if t+1 < len(Bhist) and Bhist[t+1] in lookup_df.index:
                    d_ip1 = embed_tables['newsbody'].get(
                        lookup_df.loc[Bhist[t+1]]['Head'],
                        torch.zeros_like(d_i)
                    )
                pull_term = self.W_pull(torch.tensor([[torch.dot(h_clk, d_ip1)+torch.dot(h_skp, d_i)]],
                                                     device=self.device)).squeeze(0)
                c_i = torch.tanh(self.W_skp.weight @ self.e_skp + d_i + pull_term) * h * d_i
            elif rel == "gen_summ":
                c_i = (self.W_gensumm.weight @ self.e_gensumm * h) * d_i_title
            elif rel == "summ_gen":
                gate_summgen = self.W_s(self.W_sumgen.weight @ self.e_sumgen)
                c_i = self.softmin_pool(gate_summgen * s_i, (1 - gate_summgen) * d_i)
                h_clk = h_clk + self.W_d((torch.ones_like(d_i_title) - d_i_title) * s_i)
            else:
                c_i = d_i

            # update hidden
            z_i = self.Wh(h) + self.Wc(c_i)
            p_i = torch.softmax(self.Wz(z_i) + self.b_z, dim=-1)
            m_i = p_i[0]*0.1 + p_i[1]*0.5 + p_i[2]*0.9
            if rel == "click": h_clk = h_clk + m_i * c_i
            elif rel == "skip": h_skp = h_skp * (1 - m_i) + c_i
            h = self.beta * h_clk + (1 - self.beta) * h_skp

            e_i = torch.tanh(self.W_emb(c_i) + self.b_emb)
            E_seq.append(e_i)

        if not E_seq:
            return torch.zeros(self.hidden_dim, device=self.device), None, total_loss

        # === PASS 2: contextualize (E′_seq) ===
        Eprime_seq = []
        eps = 1e-9
        for i, e_i in enumerate(E_seq):
            if i == 0:
                e_prime = e_i
            else:
                e_prev, e_prime_prev = E_seq[i-1], Eprime_seq[-1]
                theta_i = math.pi * torch.tanh(self.W_theta(torch.sigmoid(self.W_angle(e_prime_prev))))
                m_i = F.softplus(self.W_m(self.W_h(e_prime_prev)))

                v_i = (e_i - e_prime_prev) / (e_i - e_prime_prev).norm(p=2).clamp(min=eps)
                u_prev = e_prev / e_prime_prev.norm(p=2).clamp(min=eps)
                o_i = (v_i - torch.dot(v_i, u_prev) * u_prev)
                o_i = o_i / o_i.norm(p=2).clamp(min=eps)

                e_prime = e_prime_prev + m_i * (torch.cos(theta_i) * u_prev + torch.sin(theta_i) * o_i).squeeze()

            Eprime_seq.append(e_prime)

            # per-step auxiliary loss
            tail_id = lookup_df.loc[Bhist[i]]['Tail']
            if tail_id in self.tail2idx:
                logits_step = self.classifier(e_prime.unsqueeze(0))
                target_step = torch.tensor([self.tail2idx[tail_id]], device=self.device)
                total_loss = total_loss + F.cross_entropy(logits_step, target_step)

        # === Final prediction on Bpos ===
        eprime_last = Eprime_seq[-1]

        # Next-step embedding prediction
        eprime_next = self.W_next(eprime_last)

        logits_pos = None
        if Bpos in lookup_df.index:
            tail_id_pos = lookup_df.loc[Bpos]['Tail']
            if tail_id_pos in self.tail2idx:
                logits_pos = self.classifier(eprime_next.unsqueeze(0))
                target_pos = torch.tensor([self.tail2idx[tail_id_pos]], device=self.device)
                total_loss = total_loss + F.cross_entropy(logits_pos, target_pos)

        return eprime_last, eprime_next, logits_pos, total_loss


In [43]:
# -------------------------
# Inverse decoder for a single predicted embedding (d_next_pred as e'_pos)
# -------------------------
class BehaviorInverseDecoderPredict(nn.Module):
    """
    Inverse mapping that takes a single predicted e' (embedding for Bpos)
    and the head/headline embedding for Bpos, and returns:
      - c'_pos (approx pseudo-content)
      - s_hat_pos (approx summary)
    """
    def __init__(self, hidden_dim, device, debug=False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.debug = debug

        # Learnable pseudo-inverse / disentanglers
        self.W_emb_pinv = nn.Linear(hidden_dim, hidden_dim, bias=False)   # approx W_emb^+
        self.W1_pinv = nn.Linear(hidden_dim, hidden_dim, bias=False)      # remove head contribution
        self.W2_pinv = nn.Linear(hidden_dim, hidden_dim, bias=False)      # map residual -> summary

    def _show(self, name, tensor, maxlen=6):
        if not self.debug:
            return
        if isinstance(tensor, torch.Tensor):
            if tensor.ndim == 0:
                print(f"{name}: {tensor.item():.6f}")
            else:
                flat = tensor.detach().cpu().numpy().flatten()
                vals = ", ".join(f"{x:.6f}" for x in flat[:maxlen])
                if len(flat) > maxlen: vals += ", ..."
                print(f"{name} (shape={tuple(tensor.shape)}): [{vals}]")
        else:
            print(f"{name}: {tensor}")

    @staticmethod
    def atanh_safe(x, eps=1e-6):
        x = x.clamp(-1+eps, 1-eps)
        return 0.5 * torch.log((1+x) / (1-x))

    def forward(self, eprime_pos, b_emb, h_pos):
        """
        eprime_pos: tensor (hidden_dim,) -- predicted e' embedding for Bpos
        b_emb: encoder bias b_emb (tensor (hidden_dim,))
        h_pos: head/headline embedding for Bpos (tensor (hidden_dim,))
        Returns: c_prime_pos, s_hat_pos
        """
        # 1) invert embedding nonlinearity: atanh(e') - b_emb
        x = self.atanh_safe(eprime_pos) - b_emb  # (hidden_dim,)
        c_prime_pos = self.W_emb_pinv(x)         # approx c'_pos

        # 2) subtract head contribution and map to summary
        residual = c_prime_pos - self.W1_pinv(h_pos)
        s_hat_pos = self.W2_pinv(residual)

        # debug prints
        self._show("eprime_pos", eprime_pos)
        self._show("atanh(eprime_pos)-b_emb", x)
        self._show("c'_pos", c_prime_pos)
        self._show("residual (c' - W1^+ h)", residual)
        self._show("s_hat_pos", s_hat_pos)

        return c_prime_pos, s_hat_pos

In [74]:
class PersonalizedT5Summarizer(nn.Module):
    def __init__(self, hidden_dim, t5_model, behavior_encoder, inverse_decoder, device):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.t5 = t5_model.eval()
        self.behavior_encoder = behavior_encoder
        self.inverse_decoder = inverse_decoder
        self.device = device

        # Freeze T5
        for param in self.t5.parameters():
            param.requires_grad = False

        # Learnable gates and attention transforms
        self.W_prime = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.W_qry = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.W_key = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.W_val = nn.Linear(hidden_dim, hidden_dim, bias=True)

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables, sid2sum, tokenizer, max_len=50):
        # ----------------------
        # 1️⃣ Behavior Encoder
        # ----------------------
        eprime_last, eprime_next, logits_pos, enc_loss = self.behavior_encoder(
            Bhist, Bpos, lookup_df, tail2idx, embed_tables
        )
        print("✅ Behavior Encoder:")
        print(f"  eprime_last shape: {tuple(eprime_last.shape)}")
        print(f"  Encoder loss: {enc_loss.item():.4f}")

        # ----------------------
        # 2️⃣ Inverse Decoder → s_hat
        # ----------------------
        head_emb = embed_tables['headline'].get(Bpos, torch.zeros(self.hidden_dim, device=self.device))
        _, s_hat = self.inverse_decoder(eprime_last, self.behavior_encoder.b_emb, head_emb)
        print("✅ Inverse Decoder:")
        print(f"  s_hat shape: {tuple(s_hat.shape)}")
        print(f"  s_hat sample (first 6 dims): {s_hat.detach().cpu().numpy()[:6]}")

        # ----------------------
        # 3️⃣ Personalized T5 Encoder
        # ----------------------
        if Bpos not in lookup_df.index:
            print("⚠️ Bpos not in lookup_df; returning encoder loss only.")
            return enc_loss, None

        tail_id = lookup_df.loc[Bpos]['Tail']
        doc_text = nid2body.get(tail_id, " ")

        input_text = "generate headline for: " + doc_text
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(self.device)

        with torch.no_grad():
            encoder_outputs = self.t5.encoder(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"]
            )

        enc_states = encoder_outputs.last_hidden_state  # (1, seq_len, hidden_dim)
        print("✅ Raw T5 Encoder States:")
        print(f"  Shape: {enc_states.shape}")
        print(f"  First token (first 10 dims): {enc_states[0,0,:10].detach().cpu().numpy()}")

        # Apply gated eprime_last
        gated_eprime = torch.sigmoid(self.W_prime(eprime_last))  # (hidden_dim,)
        enc_states_gated = enc_states * gated_eprime.unsqueeze(0).unsqueeze(0)
        print("✅ Gated/Contextualized Encoder States:")
        print(f"  First token (first 10 dims): {enc_states_gated[0,0,:10].detach().cpu().numpy()}")

        # Inject s_hat
        key = self.W_key(s_hat).unsqueeze(0).unsqueeze(1)   # (1,1,hidden_dim)
        value = self.W_val(s_hat).unsqueeze(0).unsqueeze(1) # (1,1,hidden_dim)
        seq_len = enc_states_gated.size(1)
        key_exp = key.expand(-1, seq_len, -1)
        value_exp = value.expand(-1, seq_len, -1)
        personalized_enc_states = enc_states_gated + key_exp + value_exp
        print("✅ Personalized Encoder States (after s_hat injection):")
        print(f"  First token (first 10 dims): {personalized_enc_states[0,0,:10].detach().cpu().numpy()}")

        # ----------------------
        # 4️⃣ Decoder with gold summary
        # ----------------------
        gold_summary_text = sid2sum.get(tail_id, "")
        print("Gold summary source:",tail_id)
        print("Gold reference summaries:",gold_summary_text)
        # if gold_summary_text == "":
        #     print("⚠️ Gold summary missing; returning encoder loss only.")
        #     return enc_loss, None

        decoder_inputs = tokenizer(
            gold_summary_text, return_tensors="pt", max_length=max_len, truncation=True
        ).to(self.device)

        print("✅ Target Summary Tokens:")
        print(f"  Token IDs: {decoder_inputs.input_ids[0].detach().cpu().numpy()[:min(10, decoder_inputs.input_ids.size(1))]}")

        outputs = self.t5(
            input_ids=decoder_inputs.input_ids,
            attention_mask=decoder_inputs.attention_mask,
            encoder_outputs=(personalized_enc_states,),
            labels=decoder_inputs.input_ids
        )

        pred_tokens = torch.argmax(outputs.logits, dim=-1)
        print("✅ Predicted Summary Tokens:")
        print(f"  Token IDs: {pred_tokens[0,:min(10, pred_tokens.size(1))].detach().cpu().numpy()}")

        total_loss = enc_loss + outputs.loss
        print("✅ Losses:")
        print(f"  Generation loss (T5): {outputs.loss.item():.4f}")
        print(f"  Behavior encoder loss: {enc_loss.item():.4f}")
        print(f"  Total loss: {total_loss.item():.4f}")

        return total_loss, outputs.logits


In [80]:
# -----------------------------
# 1️⃣ Sample one row
# -----------------------------
sample_row = train_df.sample(1).iloc[0]
Bhist = literal_eval(sample_row['EHist'])   # list of history Doc IDs
Bpos = sample_row['EPos']                # the current position doc ID (or use 'Bpos' column if present)
print(f"🔹 Random Row UserID: {sample_row.UserID}")
print(f"📝 Sample Bhist: {Bhist}")
print(f"🎯 Target Bpos: {Bpos}")

# -----------------------------
# 2️⃣ Initialize models
# -----------------------------
behavior_encoder = BehaviorEncoder(hidden_dim, tail2idx, device, debug=True).to(device)
inverse_decoder = BehaviorInverseDecoderPredict(hidden_dim, device, debug=True).to(device)
personalized_model = PersonalizedT5Summarizer(hidden_dim, summarizer_model, behavior_encoder, inverse_decoder, device).to(device)

# -----------------------------
# 3️⃣ Forward pass through Behavior Encoder
# -----------------------------
eprime_last, eprime_next, logits_pos, enc_loss = behavior_encoder(
    Bhist, Bpos, lookup_df, tail2idx, embed_tables
)

print("\n✅ Behavior Encoder Output:")
print(f"  eprime_last shape: {eprime_last.shape}")
print(f"  eprime_next shape: {eprime_next.shape}")
if logits_pos is not None:
    pred_tail_idx = torch.argmax(logits_pos, dim=-1).item()
    pred_tail = idx2tail[pred_tail_idx]
    print(f"  Predicted tail from encoder: {pred_tail}")
else:
    print("  No predicted tail (Bpos not in lookup)")

# -----------------------------
# 4️⃣ Forward pass through Inverse Decoder
# -----------------------------
head_emb = embed_tables['headline'].get(Bpos, torch.zeros(hidden_dim, device=device))
c_prime, s_hat = inverse_decoder(eprime_next, behavior_encoder.b_emb, head_emb)

print("\n✅ Inverse Decoder Output:")
print(f"  c'_pos (first 6 dims): {c_prime.detach().cpu().numpy()[:6]}")
print(f"  s_hat_pos (first 6 dims): {s_hat.detach().cpu().numpy()[:6]}")

# -----------------------------
# 5️⃣ Forward pass through Personalized T5
# -----------------------------
tail_id = lookup_df.loc[Bpos]['Tail']
print("Target summary id",tail_id)
gold_summary_text = sid2sum.get(tail_id, "")

total_loss, logits = personalized_model(
    Bhist, Bpos, lookup_df, tail2idx, embed_tables, sid2sum, tokenizer, max_len=50
)

# -----------------------------
# 6️⃣ Decode predicted tokens
# -----------------------------
pred_summary = tokenizer.decode(torch.argmax(logits, dim=-1)[0], skip_special_tokens=True)
print("\n✅ Personalized T5 Output:")
print(f"  Target summary: {gold_summary_text[:300]}")
print(f"  Predicted summary: {pred_summary[:300]}")
print(f"  Total loss: {total_loss.item():.4f}")
print(f"  Encoder loss: {enc_loss.item():.4f}")


🔹 Random Row UserID: U180417_2
📝 Sample Bhist: ['E1133134', 'E1133135', 'E1133136', 'E1133137', 'E1133138', 'E1133139', 'E1133140', 'E1133141', 'E1133142', 'E1133143', 'E1133144']
🎯 Target Bpos: E1133145

✅ Behavior Encoder Output:
  eprime_last shape: torch.Size([768])
  eprime_next shape: torch.Size([768])
  Predicted tail from encoder: S-204452
eprime_pos (shape=(768,)): [0.087004, -0.036369, -0.119065, 0.031284, 0.091543, 0.012611, ...]
atanh(eprime_pos)-b_emb (shape=(768,)): [0.087225, -0.036385, -0.119632, 0.031294, 0.091800, 0.012611, ...]
c'_pos (shape=(768,)): [-0.047746, -0.034158, 0.023216, -0.038377, 0.071889, 0.035393, ...]
residual (c' - W1^+ h) (shape=(768,)): [-0.047746, -0.034158, 0.023216, -0.038377, 0.071889, 0.035393, ...]
s_hat_pos (shape=(768,)): [-0.023668, -0.039272, 0.013796, -0.015831, 0.002949, 0.016757, ...]

✅ Inverse Decoder Output:
  c'_pos (first 6 dims): [-0.04774594 -0.03415801  0.02321595 -0.03837652  0.0718895   0.03539296]
  s_hat_pos (first 6 dims)

In [81]:
sid2sum.get("S-29150", "")

'Royal reporter Katie Nicholl tells Entertainment Tonight that Markle "hasn\'t been in any hurry to get back in shape," and nor should she be!'

In [82]:
sid2sum.get("S-204452", "")

''